## Problem Statement:
### Predict whether it will rain today or not based on weather measurements.

## Coding:

In [30]:
import warnings
warnings.filterwarnings('ignore')

In [31]:
from timeit import default_timer
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb

In [32]:
rain = pd.read_csv("data/weatherAUS.csv")

In [33]:
rain.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


### Analyze target

In [34]:
rain['RainToday'].notna().sum()  #non-null target values

142199

In [35]:
rain['RainToday'].isna().sum()  #null target values

3261

In [36]:
rain['RainToday'].value_counts()  #target value distribution

No     110319
Yes     31880
Name: RainToday, dtype: int64

In [37]:
rain['RainToday'].value_counts().sum()  #cross-check

142199

### Drop features

In [38]:
rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [39]:
cols_to_drop = ["Date", "Location", "RainTomorrow", "Rainfall"]

In [40]:
rain.drop(cols_to_drop, axis=1, inplace=True)

In [41]:
missing_props = rain.isna().mean(axis=0)

In [42]:
missing_props.sort_values(ascending=False)  #drop columns with excessive null values (>40%)

Sunshine         0.480098
Evaporation      0.431665
Cloud3pm         0.408071
Cloud9am         0.384216
Pressure9am      0.103568
Pressure3pm      0.103314
WindDir9am       0.072639
WindGustDir      0.070989
WindGustSpeed    0.070555
Humidity3pm      0.030984
WindDir3pm       0.029066
Temp3pm          0.024811
RainToday        0.022419
WindSpeed3pm     0.021050
Humidity9am      0.018246
Temp9am          0.012148
WindSpeed9am     0.012148
MinTemp          0.010209
MaxTemp          0.008669
dtype: float64

In [43]:
over_threshold = missing_props[missing_props >= 0.4]

In [44]:
over_threshold.index

Index(['Evaporation', 'Sunshine', 'Cloud3pm'], dtype='object')

In [45]:
rain.drop(over_threshold.index, 
          axis=1, 
          inplace=True)

In [46]:
X = rain.drop("RainToday", axis=1)

In [47]:
y = rain['RainToday']

### Preprocessing

In [48]:
categorical_pipeline = Pipeline(
    steps=[
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("oh-encode", OneHotEncoder(handle_unknown="ignore", sparse=False)),
    ]
)

In [49]:
from sklearn.preprocessing import StandardScaler

numeric_pipeline = Pipeline(
    steps=[("impute", SimpleImputer(strategy="mean")), 
           ("scale", StandardScaler())]
)

In [50]:
cat_cols = X.select_dtypes(exclude="number").columns
num_cols = X.select_dtypes(include="number").columns

In [51]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   MinTemp        143975 non-null  float64
 1   MaxTemp        144199 non-null  float64
 2   WindGustDir    135134 non-null  object 
 3   WindGustSpeed  135197 non-null  float64
 4   WindDir9am     134894 non-null  object 
 5   WindDir3pm     141232 non-null  object 
 6   WindSpeed9am   143693 non-null  float64
 7   WindSpeed3pm   142398 non-null  float64
 8   Humidity9am    142806 non-null  float64
 9   Humidity3pm    140953 non-null  float64
 10  Pressure9am    130395 non-null  float64
 11  Pressure3pm    130432 non-null  float64
 12  Cloud9am       89572 non-null   float64
 13  Temp9am        143693 non-null  float64
 14  Temp3pm        141851 non-null  float64
dtypes: float64(12), object(3)
memory usage: 16.6+ MB


In [52]:
cat_cols

Index(['WindGustDir', 'WindDir9am', 'WindDir3pm'], dtype='object')

In [53]:
num_cols

Index(['MinTemp', 'MaxTemp', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm',
       'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am',
       'Temp9am', 'Temp3pm'],
      dtype='object')

In [54]:
X['WindDir9am'].value_counts()  #category distribution of a categorical variable

N      11758
SE      9287
E       9176
SSE     9112
NW      8749
S       8659
W       8459
SW      8423
NNE     8129
NNW     7980
ENE     7836
NE      7671
ESE     7630
SSW     7587
WNW     7414
WSW     7024
Name: WindDir9am, dtype: int64

In [55]:
full_processor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_pipeline, num_cols),
        ("categorical", categorical_pipeline, cat_cols),
    ]
)

In [56]:
X_processed = full_processor.fit_transform(X)

In [57]:
y_processed = SimpleImputer(strategy="most_frequent").fit_transform(
    y.values.reshape(-1, 1)
)

### Split data into training, evaluation and test dataset

In [58]:
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_processed, stratify=y_processed, random_state=42
)

### parameter tuning

In [55]:
param_grid = {
    "max_depth": [3, 4, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1],
    "gamma": [0, 0.25, 1],
    "reg_lambda": [0, 1, 10],
    "scale_pos_weight": [1, 3, 5],
    "subsample": [0.8],
    "colsample_bytree": [0.5],
}

In [57]:
xgb_cl = xgb.XGBClassifier(objective="binary:logistic")

In [58]:
grid_cv = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")

In [60]:
start = default_timer()
grid_cv.fit(X_processed, y_processed.ravel())
end = default_timer()
print(f'Time elapsed (hh:mm:ss.ms) {timedelta(seconds=end-start)}')

C:\Anaconda3\envs\tensorflow22_env\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Time elapsed (hh:mm:ss.ms) 5:23:07.122881


In [61]:
grid_cv.best_score_

0.8541085582312341

In [62]:
grid_cv.best_params_

{'colsample_bytree': 0.5,
 'gamma': 0.25,
 'learning_rate': 0.1,
 'max_depth': 7,
 'reg_lambda': 10,
 'scale_pos_weight': 1,
 'subsample': 0.8}

### final XGBoost classifier

In [46]:
best_params = {
    'gamma': 0.25,
    'learning_rate': 0.1,
    'max_depth': 7,
    'reg_lambda': 10,
    'scale_pos_weight': 1,
}

In [64]:
final_cl = xgb.XGBClassifier(
    **best_params,
    objective="binary:logistic",
    colsample_bytree=0.5,
    subsample=0.8
)

In [65]:
start = default_timer()
final_cl.fit(X_train, y_train.ravel())
end = default_timer()
print(f'Time elapsed (hh:mm:ss.ms) {timedelta(seconds=end-start)}')

Time elapsed (hh:mm:ss.ms) 0:00:20.046535


### Performance of XGBoost classifier

In [66]:
preds = final_cl.predict(X_test)

In [67]:
preds_prob = final_cl.predict_proba(X_test)

In [71]:
# Score
accuracy_score(y_test, preds)

0.8492506531005087

In [72]:
roc_auc_score(y_test, preds_prob[:,1])

0.8812395488187237

### Detailed performance measures

In [74]:
cross_val_score(final_cl, X_test, y_test.ravel(), scoring="roc_auc", cv = 3)

array([0.882761  , 0.87467024, 0.86644512])

In [75]:
cross_val_score(final_cl, X_test, y_test.ravel(), cv = 3)

array([0.84730243, 0.84697245, 0.84308225])

In [76]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

          No       0.87      0.95      0.91     28395
         Yes       0.72      0.50      0.59      7970

    accuracy                           0.85     36365
   macro avg       0.80      0.73      0.75     36365
weighted avg       0.84      0.85      0.84     36365



In [77]:
confusion_matrix(y_test, preds)   #confusion=[[TN,FP],[FN,TP]]

array([[26863,  1532],
       [ 3950,  4020]], dtype=int64)

In [22]:
precision = np.round(4020/(1532+4020),2); precision #precision = TP/(FP+TP)

0.72

In [23]:
recall = np.round(4020/(3950+4020),2); recall  #recall = TP/(FN+TP)

0.5

In [24]:
f1_score = np.round(2*(precision*recall)/(precision+recall),2); f1_score  #f1-score

0.59

In [25]:
26863+1532  #Actual Negative = TN+FP

28395

In [26]:
3950+4020  #Actual Positive = FN+TP

7970

In [81]:
pd.DataFrame(y_test).value_counts()  #support

No     28395
Yes     7970
dtype: int64